In [5]:
from mpmath import iv
from graphviz import Digraph

import os


class Tree_node:
    default_segment = iv.mpf([0.0, 0.0])
    #операторы. Каждый оператор - список
    #нулевой элемент списка - приоритет
    #первый - тип оператора 'VAR' для переменной, 'OO' - для унарного оператора, 'TO' - для бинарного
    #второй - лямбда-функция самой операции
    # t = (2, 'TO', lambda x, y: x + y)
    # t[0] == 2
    # t[1] == 'TO'
    # t[2] == lambda x, y: x + y
    OPERATORS = {'+': (2, 'TO', lambda x, y: x + y), '-': (2,'TO', lambda x, y: x - y),
                '*': (3, 'TO', lambda x, y: x * y), '/': (3, 'TO', lambda x, y: x / y),
                '^': (4, 'TO', lambda x, y: x ** y), 'x': (1, 'VAR', lambda: Tree_node.default_segment), 
                'sin': (5, 'OO', lambda x: iv.sin(x)), 'cos': (5, 'OO', lambda x: iv.cos(x)), 
                'ctg': (5, 'OO', lambda x: iv.cot(x)), 'tg': (5, 'OO', lambda x: iv.tan(x)),
                'log': (5, 'OO', lambda x: iv.log(x)), 'exp': (5, 'OO', lambda x: iv.exp(x))}
    
    def __init__(self, operand, left = None, right = None):
        #для красивой печати. Можно обойтись без этого
        self.name = operand
        #если в формуле число, то значение в вершине - сегмент
        if(type(operand) == float):
            #тип - чтобы отличать сегмент, операцию с одним операндом и операцию с двумя операндами
            self.type = 'segment'
            #это значение в формуле. Для числа и 'x' мы точно знаем результат
            self.value = iv.mpf(operand)
            #метка, что значение уже посчитано и пересчитывать не нужно (это для большой формулы долго)
            self.is_value_computed = True
            return
        #если в формуле 'x', то возвращаем сегмент со входа
        if(Tree_node.OPERATORS[operand][1] == 'VAR'):
            self.type = 'segment'
            self.value = Tree_node.default_segment
            self.is_value_computed = True
            return
        if(Tree_node.OPERATORS[operand][1] == 'OO'):
            self.type = 'OO'
            #для операции мы храним саму лямбда-функцию операции

            self.operation = Tree_node.OPERATORS[operand][2]
            #операнд всего один. Это вершина дерева
            self.subnode = left
            self.is_value_computed = False
            return
        if(Tree_node.OPERATORS[operand][1] == 'TO'):
            self.type = 'TO'
            self.operation = Tree_node.OPERATORS[operand][2]
            #левый и правый операнды формулы тоже вершины дерева
            self.left = left
            self.right = right
            self.is_value_computed = False
            return
    
    #получаем значение в вершине
    def get_value(self):
        #если уже посчитано, то выдаем значение сразу
        if(self.is_value_computed):
            return self.value
        #если нет - считаем значение в поддереве
        if(self.type == 'OO'):
            self.value = self.operation(self.subnode.get_value())
            self.is_value_computed = True
            return self.value
        if(self.type == 'TO'):
            self.value = self.operation(self.left.get_value(), self.right.get_value())
            self.is_value_computed = True
            return self.value
    
    def recompute_value(self):
        if(self.name == 'x'):
            return Tree_node.default_segment
        if(self.type == 'OO'):
            self.value = self.operation(self.subnode.recompute_value())
            self.is_value_computed = True
            return self.value
        if(self.type == 'TO'):
            self.value = self.operation(self.left.recompute_value(), self.right.recompute_value())
            self.is_value_computed = True
            return self.value
        return self.value

    def get_visual(self, graph, id):
        if(self.is_value_computed):
            graph.node(name = str(id), label = '%s\n%s' % (self.name, self.value.__str__()))
        else:
            graph.node(name = str(id), label = '%s' % (self.name))
        if(self.type == 'OO'):
            self.subnode.get_visual(graph, id * 2)
            graph.edge(str(id), str(id * 2))
        elif(self.type == 'TO'):
            self.left.get_visual(graph, id * 2 + 1)
            self.right.get_visual(graph, id * 2)
            graph.edge(str(id), str(id * 2))
            graph.edge(str(id), str(id * 2 + 1))
        
    
    def __str__(self):
        if(self.type == 'segment'):
            return self.value.__str__()
        elif(self.type == 'OO'):
            if(self.is_value_computed):
                return '{}::( {} = {} )'.format(self.name, self.subnode.__str__(), self.value.__str__())
            return '{}::( {} )'.format(self.name, self.subnode.__str__())
        elif(self.type == 'TO'):
            if(self.is_value_computed):
                return '{}::( {} , {} = {})'.format(self.name, self.left.__str__(), self.right.__str__(), self.value.__str__())
            return '{}::( {} , {} )'.format(self.name, self.left.__str__(), self.right.__str__())

#переводит формулу-строку в список (не совсем, просто строит генератор, но это не так важно) из элементов, запись до сих пор псевдо-инфиксная:
#'sin(',...,'tg(' переводит в ['(','sin'], ...
#переменная, число, бинарные операторы, '(' и ')' - в себя же 
def parse(formula_string):
    number = ''
    #костыль для пропуска лишних символов у операторов, запись которых больше 1 символа
  
    skip_counter = 0
    for i in range(0, len(formula_string)):
        #пропускаем лишнее
        if(skip_counter > 0):
            skip_counter -= 1
            continue
        #обрабатываем числа и скобки - они собираются посимвольно
        s = formula_string[i]
        if s in '1234567890.':
            number += s
            continue 
        # если символ не цифра, то выдаём собранное число и начинаем собирать заново
        elif number: 
            yield float(number) 
            number = ''
        if s in ('(',')'):
            yield s
            continue
        #костыль для быстродействия - все операции по длине не превышают 3
        s = formula_string[i:i+3]
        for op in Tree_node.OPERATORS:
            if s.find(op) == 0:
                #чтобы была правильная инфиксная запись проще сразу поменять 'sin' и '(' местами
                if Tree_node.OPERATORS[op][1] == 'OO':
                    yield('(')
                    skip_counter += 1
                skip_counter += len(op) - 1
                yield op
                
                break
    # если в конце строки есть число, выдаём его
    if number:
        yield float(number) 

#переводим формулу в дерево
#на выходе функции хотим получить корень дерева

def infix_to_tree(parsed_formula):
    #для хранения необработанных операций
    stack = []
    #для хранения вершин без родителя
    node_stack = []
    for token in parsed_formula:
        if token == '(':
            stack.append(token)
        elif token == ')':
            #по закрывающей скобке можно получить корень поддерева, в котором содержится всё, что в скобках 
            #log(...) + cos((...)...)
            #( log ... ) + ( cos (...) ...)
            k = stack.pop()
            while k != '(':
                if(Tree_node.OPERATORS[k][1] == 'OO'):
                    node_stack.append(Tree_node(k, node_stack.pop()))
                elif(Tree_node.OPERATORS[k][1] == 'TO'):
                    right = node_stack.pop()
                    left = node_stack.pop()
                    node_stack.append(Tree_node(k, left, right))
                k = stack.pop()
        elif token in Tree_node.OPERATORS:
            if Tree_node.OPERATORS[token][1] == 'VAR':
                node_stack.append(Tree_node(token))
            else:
                while stack and stack[-1]!='(' and Tree_node.OPERATORS[stack[-1]][1] == 'TO' and Tree_node.OPERATORS[token][0] <= Tree_node.OPERATORS[stack[-1]][0] :
                    op = stack.pop()
                    right = node_stack.pop()
                    left = node_stack.pop()
                    node_stack.append(Tree_node(op, left, right))
                stack.append(token)
        else:
            node_stack.append(Tree_node(token))
            
    while stack:
        k = stack.pop()
        if(Tree_node.OPERATORS[k][1] == 'OO'):
            node_stack.append(Tree_node(k, node_stack.pop()))
        elif(Tree_node.OPERATORS[k][1] == 'TO'):
            right = node_stack.pop()
            left = node_stack.pop()
            node_stack.append(Tree_node(k, left, right))

    return node_stack[0]

def subdiv(tree_root, c):
    Tree_node.default_segment = c
    result_segment = tree_root.recompute_value()
    if(not(result_segment in iv.mpf([0, '+inf']))):
        Tree_node.default_segment = c.a
        r_s_l = tree_root.recompute_value()
        Tree_node.default_segment = c.b
        r_s_r = tree_root.recompute_value()
        if((r_s_l in iv.mpf(['-inf', 0])) or (r_s_r in iv.mpf(['-inf', 0]))):
            return [], [c]
        else:
            p_s_l, n_s_l = subdiv(tree_root, iv.mpf([c.a, c.mid]))
            p_s_r, n_s_r = subdiv(tree_root, iv.mpf([c.mid, c.b]))
            res_p = []
            res_n = []
            for i in p_s_l:
                res_p.append(i)
            for i in p_s_r:
                res_p.append(i)
            for i in n_s_l:
                res_n.append(i)
            for i in n_s_r:
                res_n.append(i)
            return res_p, res_n
    else:
        return [c], []

formula = input()
def_seg = iv.mpf([float(i) for i in input().replace('[','').replace(']','').replace(',',' ').split()])
Tree_node.default_segment = def_seg
#получаем генератор из формулы
t = parse(formula)
#получаем дерево из генератора
p = infix_to_tree(t)
#вычисляем все значения для дерева
c = p.get_value()
#создаём объект ориентированого графа
d = Digraph('formula')
#отрисовываем все вершины и грани графа
p.get_visual(d, 1)
#отрисовываем дерево в test.pdf
d.render('test', directory = 'C:/неравенства', view=False, format='pdf')

if(not(c in iv.mpf([0, '+inf']))):
    q, v = subdiv(p, Tree_node.default_segment)
    if(v == []):
        print("positive")
        print(len(q))
    else:
        print(v[0])


cos(x)-1+x^2/2+1/1000
-2 1
positive
340
